In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('../input/dont-overfit-ii/train.csv')
df_test=pd.read_csv('../input/dont-overfit-ii/test.csv')

labels=df_train.columns.drop(['id','target'])
target=df_train['target']
ide=df_test['id']
df_test=df_test.drop('id',axis=1)

In [ ]:
df_train.info()

In [ ]:
df_train.head()

**As our training data is considerd small data and will tend to over fit simple models should be used or complex models with a good tuning the hyper parameters like L1 and L2 for regulaization and any parameter will lead to genralize our model, So I will try to Firstly see if there is any missing values first then will try simple models**

In [ ]:
msno.matrix(df_train,figsize=(20,7))

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(df_train[labels] , target,  train_size=0.8, test_size=0.2, random_state =0)

In [ ]:
# First I will try to use Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)
preds=model.predict(x_test)


In [ ]:
from sklearn.metrics import roc_auc_score
print('auc_train:', roc_auc_score(y_train,model.predict(x_train)))
print('auc_test:' , roc_auc_score(y_test, preds))

**So I will be using a more complex model and try to make a good tuning for it**

In [ ]:
from xgboost import XGBClassifier

m = XGBClassifier(max_depth=2,gamma=11,eta=0.8,reg_alpha=0.7,reg_lambda=0.9,eval_metric=None)
m.fit(x_train, y_train)
pred=m.predict(x_test)

In [ ]:
print('auc_train:', roc_auc_score(y_train,m.predict(x_train)))
print('auc_test:' , roc_auc_score(y_test, pred))

In [ ]:
print(df_test.shape)
ou=m.predict(df_test)

In [ ]:
output=pd.DataFrame({'id':ide,
                    'target':ou})

In [ ]:
output.to_csv('submission.csv', index=False)